import libraries

In [13]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from gtts import gTTS
import os
import pygame

function for audio

In [14]:
def speak_text(text):
    # Initialize pygame mixer (ensure it's initialized every time)
    if not pygame.mixer.get_init():
        pygame.mixer.init()

    tts = gTTS(text=text, lang='en')
    filename = "temp_audio.mp3"
    tts.save(filename)
    pygame.mixer.music.load(filename)
    pygame.mixer.music.play()
    while pygame.mixer.music.get_busy():  # Wait for playback to finish
        continue
    pygame.mixer.music.stop()  # Stop the music
    pygame.mixer.quit()  # Quit the mixer to release the file
    os.remove(filename)  # Delete the file

In [15]:
# Load the model
model = load_model('action.h5')

# Define actions
actions = ['hello', 'thanks', 'iloveyou']

In [16]:
# Visualization function
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60 + num * 40), (int(prob * 100), 90 + num * 40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85 + num * 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    return output_frame

In [18]:
# Initialize MediaPipe holistic model
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    # Draw landmarks
    mp_drawing.draw_landmarks(
        image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
        mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1, circle_radius=1),
        mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1, circle_radius=1)
    )
    mp_drawing.draw_landmarks(
        image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2)
    )
    mp_drawing.draw_landmarks(
        image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2)
    )
    mp_drawing.draw_landmarks(
        image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
        mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
        mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
    )

def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])


In [19]:
# Variables
sequence = []
sentence = []
predictions = []
threshold = 0.5

In [20]:
cap = cv2.VideoCapture(0)

# Set Mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]

        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            detected_action = actions[np.argmax(res)]
            print(f"Detected Action: {detected_action}")
            predictions.append(np.argmax(res))
            
            # Text-to-Speech logic
            if np.unique(predictions[-10:])[0] == np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    if len(sentence) == 0 or detected_action != sentence[-1]:
                        sentence.append(detected_action)
                        speak_text(detected_action)

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
        
        cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 903ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Detected Action: hello
1/1 ━━━━━━━━━━━━━━━━━━━